In [1]:
!pip install ollama
!pip install ragas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 796.9/796.9 kB 6.1 MB/s eta 0:00:00


In [2]:
pip install rouge_score

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 10.4 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=09c583b7478519ab677ca5e36d01807ea4455c96568ce1410761bd93f803eb51
  Stored in directory: /home/jovyan/.cache/pip/wheels/85/9d/af/01feefbe7d55ef5468796f0c68225b6788e85d9d0a281e7a70
Successfully built rouge_score
Note: you may need to restart the kernel to use updated packages.


In [3]:
!pip install bert-score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.4/906.4 MB 12.2 MB/s eta 0:00:0000:0100:02
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 24.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 20.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 23.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 17.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 21.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 29.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 29.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 25.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 27.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━

In [4]:
from ragas import evaluate
from ragas import EvaluationDataset, SingleTurnSample
from ragas.metrics import (
    RougeScore,
    StringPresence
)

In [6]:
import pandas as pd
from rouge_score import rouge_scorer
from bert_score import BERTScorer

nomeArquivoResp = "respostasLLM2024_12_12_00_08" # Substituir pelo arquivo gerado com os resultados 

dfRespostasLLM = pd.read_csv(nomeArquivoResp + ".csv")

dfRespostasLLM['gtAnswer'] = dfRespostasLLM['gtAnswer'].fillna('N/D')

listaStringPresences = []

listaBertScoreP = []
listaBertScoreR = []
listaBertScoref1 = []

listaRougeP = []
listaRougeR = []
listaRougef1 = []

# Itera sobre cada linha do DataFrame
for index, row in dfRespostasLLM.iterrows():
    
    #Metrica StringPresence
    
    sample = SingleTurnSample(
        user_input=row["question"],
        reference=row["gtAnswer"],
        response=row["modelAnswer"],
    )
    scorer = StringPresence()
    valorStringPresence = await scorer.single_turn_ascore(sample)

    listaStringPresences.append(valorStringPresence)

    #Metrica Rouge
    scorer = rouge_scorer.RougeScorer(['rouge1'], use_stemmer=True)
    scores = scorer.score(row["gtAnswer"], row["modelAnswer"])

    listaRougeP.append(scores['rouge1'].precision)
    listaRougeR.append(scores['rouge1'].recall)
    listaRougef1.append(scores['rouge1'].fmeasure)
     

    #Metrica BERTScorer
    scorer = BERTScorer(model_type='bert-base-uncased')
    precision, recall, F1 = scorer.score([row["modelAnswer"]], [row["gtAnswer"]])

    listaBertScoreP.append(precision.mean().item())
    listaBertScoreR.append(recall.mean().item())
    listaBertScoref1.append(F1.mean().item())

dfRespostasLLM['stringPresence'] = listaStringPresences

dfRespostasLLM['RougeScoreP'] = listaRougeP
dfRespostasLLM['RougeScoreR'] = listaRougeR
dfRespostasLLM['RougeScoreF1'] = listaRougef1
 
dfRespostasLLM['BertScoreP'] = listaBertScoreP
dfRespostasLLM['BertScoreR'] = listaBertScoreR
dfRespostasLLM['BertScoreF1'] = listaBertScoref1

dfRespostasLLM.to_csv(nomeArquivoResp + "_metricas.csv", index=False)  

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]